# In this file we create an ANN model ( testing Purpose ) and fine tune this model for production and deployment.

In [18]:
##  Import Libraries

import pandas as pd
from keras.src.models import Sequential
from keras.src.layers import Dense , Dropout
from keras.src.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

import warnings
warnings.filterwarnings("ignore")

In [13]:
## Spliting data into train and test data

df = pd.read_csv('../Data/Clean_Raw_Data')

X=df.iloc[:,:-1]
y=df['NObeyesdad']

x_train,  x_test, y_train , y_test =train_test_split(X,y, test_size=0.2, random_state=42)

x_train.shape,  x_test.shape, y_train.shape , y_test.shape

((1688, 16), (423, 16), (1688,), (423,))

In [23]:
## One-Hot encoding for target vaariable
y_test = to_categorical(y_test, 7)
y_train = to_categorical(y_train, 7)

In [24]:
## Scaling the data by using StandardScaler


scaler = StandardScaler()

x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [25]:
## Building the ANN model

from turtle import mode


model=Sequential()

model.add(Dense(128, activation='relu', input_shape=(x_train_scaled.shape[1],)))
model.add(Dropout(0.2))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(16, activation='relu'))

## Output Layer.
model.add(Dense(7, activation='softmax'))


# Compile the Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [26]:
## Train the model

history = model.fit(x_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2)


Epoch 1/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.2395 - loss: 1.9131 - val_accuracy: 0.3994 - val_loss: 1.6283
Epoch 2/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3876 - loss: 1.6012 - val_accuracy: 0.5621 - val_loss: 1.2300
Epoch 3/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5510 - loss: 1.2562 - val_accuracy: 0.6598 - val_loss: 0.9047
Epoch 4/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6073 - loss: 0.9909 - val_accuracy: 0.7041 - val_loss: 0.7035
Epoch 5/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6898 - loss: 0.8199 - val_accuracy: 0.7751 - val_loss: 0.5676
Epoch 6/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7177 - loss: 0.7169 - val_accuracy: 0.8373 - val_loss: 0.4835
Epoch 7/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7324 - loss: 0.6462 - val_accuracy: 0.8550 - val_loss: 0.4146
Epoch 8/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7630 - loss: 0.5699 - val_accuracy: 0.8757 - val_loss

In [27]:
test_loss, test_accuracy = model.evaluate(x_test_scaled, y_test)

print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9185 - loss: 0.1930 
Test Loss: 0.1801084578037262
Test Accuracy: 0.9314420819282532
